In [2]:
import sys
sys.path.append('..')

import os
import numpy as np
from sklearn.model_selection import train_test_split
from src.dataset import BrainMRIDataset

# Seting data path
DATA_PATH = '../data/mri-segmentation/kaggle_3m/'

In [3]:
# Get all patient IDs
dataset = BrainMRIDataset(data_path=DATA_PATH)
patient_ids = list(set([os.path.basename(os.path.dirname(path)) for path in dataset.image_paths]))

# Split patients
train_patients, test_patients = train_test_split(patient_ids, test_size=0.15, random_state=42)
train_patients, val_patients = train_test_split(train_patients, test_size=0.176, random_state=42)

# Create filtered datasets
train_dataset = BrainMRIDataset(DATA_PATH, patient_list=train_patients)
val_dataset = BrainMRIDataset(DATA_PATH, patient_list=val_patients)
test_dataset = BrainMRIDataset(DATA_PATH, patient_list=test_patients)

print(f"Train dataset: {len(train_dataset)} slices")
print(f"Val dataset: {len(val_dataset)} slices")
print(f"Test dataset: {len(test_dataset)} slices")

Found 3929 image-mask pairs
Found 2873 image-mask pairs
Found 532 image-mask pairs
Found 524 image-mask pairs
Train dataset: 2873 slices
Val dataset: 532 slices
Test dataset: 524 slices
